<a href="https://colab.research.google.com/github/viniciusrdo/workshop-engdados/blob/main/WorkshopEngDados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
# Fazendo download
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz

# Descompactando os arquivos
!tar xf spark-3.1.2-bin-hadoop2.7.tgz

In [ ]:
# Configurando as variáveis de ambiente para o Java e Spark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

In [ ]:
# Instalando biblioteca do buscador e buscando a instalação do Spark
!pip install -q findspark
import findspark
findspark.init()
findspark.find()

In [ ]:
# Importando e criando a SparkSession
# a SparkSession é o nosso ponto de entrada para as APIs do framework Spark

from pyspark.sql import SparkSession
sc = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

# Printando as informações da session criada
sc      

In [ ]:
# Fazendo download do arquivo
!wget --verbose --show-progress --no-check-certificate https://raw.githubusercontent.com/jonates/opendata/master/receita_federal/receita_federal_arrecadacao_por_UF_2020.csv

In [ ]:
# carregando um conjunto de dados que baixamos da internet
receitafederal = sc.read.csv(
    path = "/content/receita_federal_arrecadacao_por_UF_2020.csv", 
    inferSchema = True, 
    header = True,
    sep = ';', 
    encoding = "UTF-8")

In [ ]:
# Verificando o tipo de objeto criado
type(receitafederal)

In [ ]:
# Espiando o dataset
receitafederal.show()

In [ ]:
# Verificando o schema() deste sparkdataframe
receitafederal.printSchema()

In [ ]:
# importando os métodos com funções para transformações de variáveis
from pyspark.sql.functions import *

In [ ]:
# Transformando o atributo irpf em numerica
receitafederal = receitafederal.withColumn(
    colName = 'irpf', 
    col = regexp_replace('irpf',',','.').cast('float')
    )

# Inspecionando o resultado
receitafederal.select('irpf').printSchema()

root
 |-- irpf: float (nullable = true)



In [ ]:
# Verificando o total do irpf por Região do Brasil
receitafederal.groupBy('regiao').sum('irpf').orderBy('regiao').show()